In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from catboost import CatBoostClassifier, Pool

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# Define the competition scorer
def competition_scorer(y_true, y_pred):
    return log_loss(y_true, y_pred, sample_weight=10**y_true)

# Information about the test

In this test, the `requests` dataset contains information about the requests made by group of individuals (or family) to the french emergency housing public service. A sample of the `requests` dataset corresponds to a unique request.

The goal is to predict the categorical variable `granted_number_of_nights` which represents the number of nights of emergency housing granted to a group. You can train your model on the `train_requests`, the predictions should be made for requests listed in the `test_requests` dataset.

The evaluation metric is given by the `competition_scorer` defined above. It corresponds to a weighted log-loss with weights 1, 10, 100, or 1000 if the `granted_number_of_nights` takes the value 0, 1, 2, or 3 respectively. Thus beware that you will be penalized harder for classification mistakes made on the higher labels.

Good luck!

## Data description

### File requests.zip

| Field name                       | Description                                                                                                                                                        | Type                     |
|----------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------|
| request\_id                      | "the id of request, which is unique in this dataset\."                                                                                                             | string                   |
| animal\_presence                 | flage indicating whether an animal is accompanying the group                                                                                                       | boolean                  |
| answer\_creation\_date           | the timestamp when the anwer was registered                                                                                                                        | timestamp \(iso format\) |
| child\_situation                 | the number representing the situation of the children in the group \(if any\)                                                                                      | integer                  |
| child\_to\_come                  | flag indicating whether the group is expecting a baby                                                                                                              | boolean                  |
| district                         | the id of the administrative district where the request is made\. Each district has its own emergency center responsible for receiving and managing the requests\. | integer                  |
| granted\_number\_of\_nights      | "the target \(train dataset only\), the categorized number of nights granted to the request"                                                                       | integer                  |
| group\_composition\_id           | the number representing the group composition                                                                                                                      | integer                  |
| group\_composition\_label        | the description of the group composition                                                                                                                           | string                   |
| group\_creation\_date            | the timestamp when the group was created in the database                                                                                                           | timestamp \(iso format\) |
| group\_id                        | "the id of the group\. A group can be linked to several requests, so not unique"                                                                                   | string                   |
| group\_main\_requester\_id       | the id of the individual making the request\. it cannot be joined with individual\_id                                                                              | string                   |
| group\_type                      | whether the group is a made of several people \(GROUP\) or just one person \(PERSON\)                                                                              | string                   |
| housing\_situation\_id           | the id of the housing situation of the group when making the request                                                                                               | integer                  |
| housing\_situation\_label        | the description of the housing situation of the group when making the request                                                                                      | string                   |
| long\_term\_housing\_request     | flag indicating whether the groupe has filed a request for a long term housing solution\.                                                                          | boolean                  |
| number\_of\_underage             | the number of individual under 18 in the group                                                                                                                     | integer                  |
| request\_backoffice\_creator\_id | the id of the emergency call center employee who received the request                                                                                              | string                   |
| request\_creation\_date          | the timestamp when the request was made\.                                                                                                                          | timestamp \(iso format\) |
| requester\_type                  | the type of person who called the emergency service \(a user / a social patrol / a third party\)                                                                   | string                   |
| social\_situation\_id            | the id representing the social situation of the group in the database                                                                                              | string                   |
| town                             | the town where the requester is living                                                                                                                             | string                   |
| victim\_of\_violence             | flag indicating whether the group includes an individual victim of violence                                                                                        | boolean                  |
| victim\_of\_violence\_type       | type of the individual victim of violence \(and if it is a case of domestic violence\)                                                                             | string                   |


### File individuals.zip

| Field name                      | Description                                                                                                               | Type                     |
|---------------------------------|---------------------------------------------------------------------------------------------------------------------------|--------------------------|
| request\_id                     | the id of the request\. It is not unique in this dataset \(a request can be for several individual in case of a group\)\. | string                   |
| individual\_id                  | the id of the individual                                                                                                  | string                   |
| birth\_month                    | month of birth of the individual                                                                                          | integer                  |
| birth\_year                     | the year of birth of the individual                                                                                       | integer                  |
| childcare\_center\_supervision  | flag representing the fact that a child receives follow\-up care                                                          | boolean                  |
| disabled\_worker\_certification | flag representing the fact that an individual is considered disabled regarding the employement law                        | boolean                  |
| gender                          | the gender of the individual                                                                                              | boolean                  |
| housing\_situation\_2\_id       | housing situation category number of the group when making the request \(more precise\)                                   | integer                  |
| housing\_situation\_2\_label    | description of the housing situation of the group when making the request \(more precise\)                                | string                   |
| individual\_creation\_date      | timestamp of the individual creation in the database                                                                      | timestamp \(iso format\) |
| individual\_role                | description of the role of the individual                                                                                 | string                   |
| individual\_role\_2\_id         | category number of the role of the individual                                                                             | integer                  |
| individual\_role\_2\_label      | "description of the role of the individual, more precise than the former"                                                 | string                   |
| marital\_status\_id             | id of the marital status of the individual                                                                                | integer                  |
| marital\_status\_label          | description of the marital status of the individual                                                                       | string                   |
| pregnancy                       | flag representing whether the individual is pregnant \(if applicable\)                                                    | boolean                  |


# Load the datasets

## Requests

In [3]:
# Load train
train = pd.read_csv(filepath_or_buffer='requests/data/train_requests.csv',
                    sep=',',
                    low_memory=False,
                    error_bad_lines=False)
# Load test
test = pd.read_csv(filepath_or_buffer='requests/data/test_requests.csv',
                   sep=',',
                   low_memory=False,
                   error_bad_lines=False)


# Rename target
train.rename(columns={'granted_number_of_nights': 'target'},
             inplace=True)

# Create target in test df before train+test concatenation
test['target'] = np.nan

requests = pd.concat((train[train.columns], test[train.columns]), axis=0)
del train, test

b'Skipping line 31303: expected 24 fields, saw 49\nSkipping line 75954: expected 24 fields, saw 49\n'


## Individuals

In [4]:
# Load train
train = pd.read_csv(filepath_or_buffer='individuals/data/train_individuals.csv',
                    sep=',',
                    low_memory=False,
                    error_bad_lines=False)
# Load test
test = pd.read_csv(filepath_or_buffer='individuals/data/test_individuals.csv',
                   sep=',',
                   low_memory=False,
                   error_bad_lines=False)

individuals = pd.concat((train[train.columns], test[train.columns]), axis=0)
del train, test

In [5]:
# # Train sample
# requests = pd.read_csv('requests/data/train_requests.csv', sep=',', low_memory=False, error_bad_lines=False)

# # Test sample
# individuals = pd.read_csv('individuals/data/train_individuals.csv', sep=',', low_memory=False, error_bad_lines=False)

# target = 'granted_number_of_nights'

# Data preparation

## Feature creation individuals

In [6]:
individuals['childcare_center_supervision_f'] = (individuals['childcare_center_supervision'] == 'f')*1
individuals['childcare_center_supervision_t'] = (individuals['childcare_center_supervision'] == 't')*1

individuals['disabled_worker_certification_f'] = (individuals['disabled_worker_certification'] == 'f')*1
individuals['disabled_worker_certification_t'] = (individuals['disabled_worker_certification'] == 't')*1

individuals['gender_female'] = (individuals['gender'] == 'female')*1

individuals['individual_creation_date'] = pd.to_datetime(individuals['individual_creation_date'], format='%Y-%m-%d')

individuals['marital_status_label_single'] = (individuals['marital_status_label'] == 'single person')*1
individuals['marital_status_label_married'] = (individuals['marital_status_label'] == 'married')*1
individuals['marital_status_label_cohabitation'] = (individuals['marital_status_label'] == 'cohabitation')*1
individuals['marital_status_label_separated'] = (individuals['marital_status_label'] == 'separated')*1
individuals['marital_status_label_marital_relationship'] = (individuals['marital_status_label'] == 'marital relationship')*1
individuals['marital_status_label_divorced'] = (individuals['marital_status_label'] == 'divorced')*1
individuals['marital_status_label_widowed'] = (individuals['marital_status_label'] == 'widowed')*1
individuals['marital_status_label_civil_partner'] = (individuals['marital_status_label'] == 'civil partner')*1

individuals['pregnancy_t'] = (individuals['pregnancy'] == 't')*1

## Aggregation and merge

In [7]:
%%time
individuals = individuals.groupby('request_id').agg(
    nb_individuals=('individual_id', 'count'),
    birth_younger=('birth_year', 'max'),
    birth_older=('birth_year', 'min'),
    nb_childcare_center_supervision_f=('childcare_center_supervision_f', 'sum'),
    nb_childcare_center_supervision_t=('childcare_center_supervision_t', 'sum'),
    nb_disabled_worker_certification_f=('disabled_worker_certification_f', 'sum'),
    nb_disabled_worker_certification_t=('disabled_worker_certification_t', 'sum'),
    nb_gender_female=('gender_female', 'sum'),
    min_individual_creation_date=('individual_creation_date', 'min'),
    max_individual_creation_date=('individual_creation_date', 'max'),
    nb_marital_status_label_single=('marital_status_label_single', 'sum'),
    nb_marital_status_label_married=('marital_status_label_married', 'sum'),
    nb_marital_status_label_cohabitation=('marital_status_label_cohabitation', 'sum'),
    nb_marital_status_label_separated=('marital_status_label_separated', 'sum'),
    nb_marital_status_label_marital_relationship=('marital_status_label_marital_relationship', 'sum'),
    nb_marital_status_label_divorced=('marital_status_label_divorced', 'sum'),
    nb_marital_status_label_widowed=('marital_status_label_widowed', 'sum'),
    nb_marital_status_label_civil_partner=('marital_status_label_civil_partner', 'sum'),
    nb_pregnancy_t=('pregnancy_t', 'sum'),
)

requests = requests.merge(individuals, on='request_id', how='left')

CPU times: user 3.92 s, sys: 198 ms, total: 4.12 s
Wall time: 4.12 s


## Feature creation requests

In [8]:
date_cols = ['answer_creation_date', 'request_creation_date', 'min_individual_creation_date',
             'max_individual_creation_date', 'group_creation_date']
for col in date_cols:
    requests[col] = pd.to_datetime(requests[col])

In [9]:
requests['delai_answer'] = (requests['answer_creation_date'] - requests['request_creation_date']).dt.days
requests['delai_group_creation_date'] = (
    requests['request_creation_date'] - requests['group_creation_date']
).dt.days
requests['delai_min_individual_creation_date'] = (
    requests['request_creation_date'] - requests['min_individual_creation_date']
).dt.days
requests['delai_max_individual_creation_date'] = (
    requests['request_creation_date'] - requests['max_individual_creation_date']
).dt.days
requests['age_younger'] = requests['request_creation_date'].dt.year - requests['birth_younger']
requests['age_older'] = requests['request_creation_date'].dt.year - requests['birth_older']
requests['freq_group_id'] = requests['group_id'].map(requests.group_id.value_counts(dropna=False))
requests['freq_backoffice_creator_id'] = requests['request_backoffice_creator_id'].map(requests.request_backoffice_creator_id.value_counts(dropna=False))
requests['freq_social_situation_id'] = requests['social_situation_id'].map(requests.social_situation_id.value_counts(dropna=False))
requests['freq_town'] = requests['town'].map(requests.town.value_counts(dropna=False))
requests['missing_count'] = requests.isnull().sum(axis=1)

In [10]:
cols_to_drop = ['group_composition_label', 'group_creation_date', 'group_id', 'group_main_requester_id',
                'housing_situation_label', 'answer_creation_date', 'birth_younger', 'birth_older',
                'min_individual_creation_date', 'max_individual_creation_date']

requests.drop(cols_to_drop, axis=1, inplace=True)

## Preprocessing

### Categoricals

In [11]:
cat_cols = ['child_situation', 'district', 'group_composition_id', 'housing_situation_id']
cat_cols.extend(requests.select_dtypes(include='object').columns)
cat_cols.remove('request_id')

for col in cat_cols:
    encoder = LabelEncoder()
    requests[col] = encoder.fit_transform(requests[col].astype('str'))

### Dates

In [12]:
date_cols = ['request_creation_date']

# date features
def create_date_features(df, column):
    df[column] = pd.to_datetime(df[column])
    df['day_{}'.format(column)] = df[column].dt.day
    df['week_{}'.format(column)] = df[column].dt.week
    df['hour_{}'.format(column)] = df[column].dt.hour
    df['weekday_{}'.format(column)] = df[column].dt.weekday
    return df

for date_col in date_cols:
    requests[date_col] = create_date_features(requests, date_col)
    requests.drop(date_col, axis=1, inplace=True)
    cat_cols.extend([f'day_{date_col}', f'week_{date_col}', f'hour_{date_col}', f'weekday_{date_col}'])

In [13]:
train = requests[requests.target.notnull()].reset_index(drop=True)
test  = requests[requests.target.isnull()].reset_index(drop=True)
features = [feat for feat in train.columns if feat not in ['request_id','target']]
del requests

## Model

In [14]:
N_Splits = 10
n_classes = 4
SEED = 999
verbose = 5
patience = 20
target = 'target'

In [15]:
params = {
    'use_best_model': True,
    'task_type': 'CPU',
    'eval_metric': 'MultiClass',
    'n_estimators': 10000,
    'loss_function': 'MultiClass',
    'random_seed': SEED
}

In [16]:
oof_pred = np.zeros((len(train), n_classes))
y_pred = np.zeros((len(test), n_classes))

skf = StratifiedKFold(n_splits=N_Splits, shuffle=True, random_state=SEED)
for fold, (tr_ind, val_ind) in enumerate(skf.split(train, train[target])):
    X_train, X_val = train[features].iloc[tr_ind], train[features].iloc[val_ind]
    y_train, y_val = train[target].iloc[tr_ind], train[target].iloc[val_ind]
    train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols, weight=10**y_train)
    val_data = Pool(data=X_val, label=y_val, cat_features=cat_cols, weight=10**y_val)
    clf = CatBoostClassifier(**params)
    clf.fit(train_data,
            eval_set=[val_data],
            early_stopping_rounds=patience,
            verbose=verbose)
    val_pred = clf.predict_proba(X_val)
    oof_pred[val_ind] = val_pred
    y_pred += clf.predict_proba(test[features]) / (N_Splits)
np.save(f'scores/oof_pred.npy',oof_pred)
np.save(f'scores/y_pred.npy',    y_pred)

0:	learn: 1.3459259	test: 1.3460110	best: 1.3460110 (0)	total: 1.22s	remaining: 3h 22m 51s
5:	learn: 1.1832475	test: 1.1819721	best: 1.1819721 (5)	total: 7.08s	remaining: 3h 16m 33s
10:	learn: 1.0684144	test: 1.0667105	best: 1.0667105 (10)	total: 14.5s	remaining: 3h 39m 28s
15:	learn: 0.9773640	test: 0.9745683	best: 0.9745683 (15)	total: 22.4s	remaining: 3h 52m 49s
20:	learn: 0.9067519	test: 0.9037813	best: 0.9037813 (20)	total: 29.8s	remaining: 3h 56m 13s
25:	learn: 0.8548155	test: 0.8510552	best: 0.8510552 (25)	total: 36.5s	remaining: 3h 53m 8s
30:	learn: 0.8089561	test: 0.8049949	best: 0.8049949 (30)	total: 43s	remaining: 3h 50m 37s
35:	learn: 0.7676664	test: 0.7634102	best: 0.7634102 (35)	total: 50s	remaining: 3h 50m 46s
40:	learn: 0.7370051	test: 0.7327623	best: 0.7327623 (40)	total: 56.6s	remaining: 3h 49m 19s
45:	learn: 0.7067146	test: 0.7017857	best: 0.7017857 (45)	total: 1m 3s	remaining: 3h 50m 28s
50:	learn: 0.6855341	test: 0.6806159	best: 0.6806159 (50)	total: 1m 10s	remaini

435:	learn: 0.4754434	test: 0.4641875	best: 0.4641875 (435)	total: 9m 25s	remaining: 3h 26m 49s
440:	learn: 0.4742203	test: 0.4632087	best: 0.4632087 (440)	total: 9m 32s	remaining: 3h 26m 38s
445:	learn: 0.4723963	test: 0.4614830	best: 0.4614830 (445)	total: 9m 39s	remaining: 3h 26m 59s
450:	learn: 0.4705856	test: 0.4599946	best: 0.4599946 (450)	total: 9m 47s	remaining: 3h 27m 25s
455:	learn: 0.4693622	test: 0.4586665	best: 0.4586665 (455)	total: 9m 54s	remaining: 3h 27m 22s
460:	learn: 0.4674060	test: 0.4571197	best: 0.4571197 (460)	total: 10m 1s	remaining: 3h 27m 25s
465:	learn: 0.4651300	test: 0.4548267	best: 0.4548267 (465)	total: 10m 9s	remaining: 3h 27m 57s
470:	learn: 0.4634030	test: 0.4534583	best: 0.4534583 (470)	total: 10m 18s	remaining: 3h 28m 30s
475:	learn: 0.4614057	test: 0.4516736	best: 0.4516736 (475)	total: 10m 26s	remaining: 3h 28m 52s
480:	learn: 0.4587417	test: 0.4496531	best: 0.4496531 (480)	total: 10m 33s	remaining: 3h 29m
485:	learn: 0.4568131	test: 0.4476583	bes

860:	learn: 0.3870582	test: 0.3995346	best: 0.3995346 (860)	total: 20m 54s	remaining: 3h 41m 52s
865:	learn: 0.3865504	test: 0.3993234	best: 0.3993234 (865)	total: 21m 1s	remaining: 3h 41m 47s
870:	learn: 0.3860643	test: 0.3989838	best: 0.3989838 (870)	total: 21m 9s	remaining: 3h 41m 47s
875:	learn: 0.3854454	test: 0.3988781	best: 0.3988781 (875)	total: 21m 17s	remaining: 3h 41m 48s
880:	learn: 0.3848432	test: 0.3986060	best: 0.3986060 (880)	total: 21m 25s	remaining: 3h 41m 49s
885:	learn: 0.3843263	test: 0.3984341	best: 0.3984341 (885)	total: 21m 33s	remaining: 3h 41m 50s
890:	learn: 0.3837419	test: 0.3981757	best: 0.3981757 (890)	total: 21m 42s	remaining: 3h 41m 52s
895:	learn: 0.3833176	test: 0.3979974	best: 0.3979974 (895)	total: 21m 50s	remaining: 3h 41m 53s
900:	learn: 0.3827694	test: 0.3976465	best: 0.3976389 (899)	total: 21m 58s	remaining: 3h 41m 59s
905:	learn: 0.3821800	test: 0.3975128	best: 0.3975128 (905)	total: 22m 7s	remaining: 3h 42m 3s
910:	learn: 0.3817663	test: 0.3974

1285:	learn: 0.3513531	test: 0.3858019	best: 0.3857394 (1283)	total: 32m 24s	remaining: 3h 39m 36s
1290:	learn: 0.3510023	test: 0.3857185	best: 0.3857185 (1290)	total: 32m 33s	remaining: 3h 39m 36s
1295:	learn: 0.3507246	test: 0.3856064	best: 0.3856064 (1295)	total: 32m 41s	remaining: 3h 39m 31s
1300:	learn: 0.3503547	test: 0.3854169	best: 0.3854108 (1299)	total: 32m 48s	remaining: 3h 39m 23s
1305:	learn: 0.3500345	test: 0.3854366	best: 0.3853895 (1302)	total: 32m 56s	remaining: 3h 39m 18s
1310:	learn: 0.3496813	test: 0.3853282	best: 0.3853282 (1310)	total: 33m 4s	remaining: 3h 39m 16s
1315:	learn: 0.3494951	test: 0.3852821	best: 0.3852153 (1313)	total: 33m 12s	remaining: 3h 39m 9s
1320:	learn: 0.3492027	test: 0.3850755	best: 0.3850755 (1320)	total: 33m 20s	remaining: 3h 39m 6s
1325:	learn: 0.3489229	test: 0.3847821	best: 0.3847821 (1325)	total: 33m 29s	remaining: 3h 39m 5s
1330:	learn: 0.3483898	test: 0.3845260	best: 0.3845260 (1330)	total: 33m 37s	remaining: 3h 39m 1s
1335:	learn: 0.

305:	learn: 0.4927239	test: 0.4819098	best: 0.4819098 (305)	total: 5m 45s	remaining: 3h 2m 28s
310:	learn: 0.4923209	test: 0.4816228	best: 0.4816228 (310)	total: 5m 51s	remaining: 3h 2m 17s
315:	learn: 0.4917774	test: 0.4812786	best: 0.4812786 (315)	total: 5m 56s	remaining: 3h 2m 11s
320:	learn: 0.4904796	test: 0.4801782	best: 0.4801782 (320)	total: 6m 2s	remaining: 3h 2m 12s
325:	learn: 0.4896224	test: 0.4795473	best: 0.4795473 (325)	total: 6m 6s	remaining: 3h 1m 19s
330:	learn: 0.4887807	test: 0.4786641	best: 0.4786641 (330)	total: 6m 13s	remaining: 3h 1m 48s
335:	learn: 0.4877522	test: 0.4778809	best: 0.4778809 (335)	total: 6m 19s	remaining: 3h 2m 7s
340:	learn: 0.4869327	test: 0.4773093	best: 0.4773093 (340)	total: 6m 24s	remaining: 3h 1m 44s
345:	learn: 0.4862672	test: 0.4767206	best: 0.4767205 (344)	total: 6m 30s	remaining: 3h 1m 42s
350:	learn: 0.4851152	test: 0.4759723	best: 0.4759723 (350)	total: 6m 37s	remaining: 3h 2m 11s
355:	learn: 0.4840075	test: 0.4753052	best: 0.4753052

...

1010:	learn: 0.3661577	test: 0.3959975	best: 0.3959975 (1010)	total: 24m 38s	remaining: 3h 39m 6s
1015:	learn: 0.3658003	test: 0.3958523	best: 0.3958523 (1015)	total: 24m 46s	remaining: 3h 39m 7s
1020:	learn: 0.3655006	test: 0.3958047	best: 0.3958047 (1020)	total: 24m 54s	remaining: 3h 39m 5s
1025:	learn: 0.3650200	test: 0.3955777	best: 0.3955777 (1025)	total: 25m 2s	remaining: 3h 39m 4s
1030:	learn: 0.3644571	test: 0.3951870	best: 0.3951700 (1029)	total: 25m 10s	remaining: 3h 39m 2s
1035:	learn: 0.3641197	test: 0.3951259	best: 0.3951259 (1035)	total: 25m 18s	remaining: 3h 39m 2s
1040:	learn: 0.3636482	test: 0.3948095	best: 0.3948095 (1040)	total: 25m 26s	remaining: 3h 39m
1045:	learn: 0.3632289	test: 0.3946011	best: 0.3945995 (1044)	total: 25m 35s	remaining: 3h 39m 1s
1050:	learn: 0.3628272	test: 0.3943532	best: 0.3943532 (1050)	total: 25m 43s	remaining: 3h 38m 58s
1055:	learn: 0.3624845	test: 0.3941091	best: 0.3941091 (1055)	total: 25m 50s	remaining: 3h 38m 54s
1060:	learn: 0.3620287

1430:	learn: 0.3342261	test: 0.3830559	best: 0.3830559 (1430)	total: 35m 59s	remaining: 3h 35m 31s
1435:	learn: 0.3340031	test: 0.3830341	best: 0.3830087 (1433)	total: 36m 7s	remaining: 3h 35m 26s
1440:	learn: 0.3337551	test: 0.3829451	best: 0.3829068 (1438)	total: 36m 16s	remaining: 3h 35m 25s
1445:	learn: 0.3333718	test: 0.3829278	best: 0.3829005 (1444)	total: 36m 24s	remaining: 3h 35m 22s
1450:	learn: 0.3331534	test: 0.3828670	best: 0.3828561 (1449)	total: 36m 32s	remaining: 3h 35m 18s
1455:	learn: 0.3329742	test: 0.3828483	best: 0.3828188 (1453)	total: 36m 40s	remaining: 3h 35m 15s
1460:	learn: 0.3326403	test: 0.3828475	best: 0.3828188 (1453)	total: 36m 48s	remaining: 3h 35m 8s
1465:	learn: 0.3323851	test: 0.3827832	best: 0.3827832 (1465)	total: 36m 56s	remaining: 3h 35m 5s
1470:	learn: 0.3320905	test: 0.3828042	best: 0.3827494 (1466)	total: 37m 4s	remaining: 3h 34m 58s
1475:	learn: 0.3317492	test: 0.3827274	best: 0.3826759 (1474)	total: 37m 12s	remaining: 3h 34m 55s
1480:	learn: 0

300:	learn: 0.4955047	test: 0.4968044	best: 0.4968044 (300)	total: 6m 25s	remaining: 3h 26m 51s
305:	learn: 0.4952071	test: 0.4964977	best: 0.4964977 (305)	total: 6m 27s	remaining: 3h 24m 44s
310:	learn: 0.4948539	test: 0.4963010	best: 0.4963010 (310)	total: 6m 34s	remaining: 3h 24m 40s
315:	learn: 0.4943184	test: 0.4958744	best: 0.4958743 (314)	total: 6m 39s	remaining: 3h 23m 53s
320:	learn: 0.4935861	test: 0.4953439	best: 0.4953439 (320)	total: 6m 46s	remaining: 3h 24m 15s
325:	learn: 0.4922592	test: 0.4940966	best: 0.4940966 (325)	total: 6m 52s	remaining: 3h 24m 8s
330:	learn: 0.4914998	test: 0.4936095	best: 0.4936095 (330)	total: 6m 57s	remaining: 3h 23m 19s
335:	learn: 0.4912454	test: 0.4933908	best: 0.4933908 (335)	total: 7m 2s	remaining: 3h 22m 32s
340:	learn: 0.4907348	test: 0.4930596	best: 0.4930596 (340)	total: 7m 7s	remaining: 3h 21m 50s
345:	learn: 0.4903560	test: 0.4927119	best: 0.4927119 (345)	total: 7m 14s	remaining: 3h 21m 50s
350:	learn: 0.4900841	test: 0.4924763	best:

730:	learn: 0.3946015	test: 0.4282657	best: 0.4282657 (730)	total: 17m 4s	remaining: 3h 36m 33s
735:	learn: 0.3938309	test: 0.4279486	best: 0.4279486 (735)	total: 17m 13s	remaining: 3h 36m 44s
740:	learn: 0.3931121	test: 0.4276609	best: 0.4276609 (740)	total: 17m 21s	remaining: 3h 36m 49s
745:	learn: 0.3925064	test: 0.4273584	best: 0.4273584 (745)	total: 17m 29s	remaining: 3h 36m 57s
750:	learn: 0.3917491	test: 0.4270116	best: 0.4270116 (750)	total: 17m 37s	remaining: 3h 37m 4s
755:	learn: 0.3910382	test: 0.4266739	best: 0.4266739 (755)	total: 17m 45s	remaining: 3h 37m 10s
760:	learn: 0.3901079	test: 0.4262957	best: 0.4262957 (760)	total: 17m 53s	remaining: 3h 37m 11s
765:	learn: 0.3894126	test: 0.4259221	best: 0.4259221 (765)	total: 18m 1s	remaining: 3h 37m 12s
770:	learn: 0.3887834	test: 0.4257428	best: 0.4257428 (770)	total: 18m 9s	remaining: 3h 37m 23s
775:	learn: 0.3882696	test: 0.4255194	best: 0.4255194 (775)	total: 18m 17s	remaining: 3h 37m 29s
780:	learn: 0.3876752	test: 0.4251

1155:	learn: 0.3517068	test: 0.4108139	best: 0.4107987 (1154)	total: 28m 36s	remaining: 3h 38m 49s
1160:	learn: 0.3513354	test: 0.4108636	best: 0.4107987 (1154)	total: 28m 44s	remaining: 3h 38m 45s
1165:	learn: 0.3510669	test: 0.4108778	best: 0.4107987 (1154)	total: 28m 51s	remaining: 3h 38m 40s
1170:	learn: 0.3508631	test: 0.4108257	best: 0.4107987 (1154)	total: 28m 59s	remaining: 3h 38m 37s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.4107987224
bestIteration = 1154

Shrink model to first 1155 iterations.


In [17]:
score = competition_scorer(train.target, oof_pred)
score

0.3919679117104138

## Submission

In [18]:
predictions = pd.concat([test['request_id'], pd.DataFrame(y_pred)], axis=1)
predictions.to_csv(f'submissions/CV_catboost_{score}.csv', index=False)